In [1]:
!pip install kora -q
from kora import drive
drive.link_nbs()

In [2]:
import utils

importing Jupyter notebook from /nbs/utils.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
StopIteration exception: 
length data_files:  85
len data_list :  85


In [17]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from keras.applications.efficientnet import EfficientNetB0
from keras.models import Model, Sequential
from keras.layers import TimeDistributed, Input, MaxPooling3D, LSTM, Dense, Conv3D, Conv2D, BatchNormalization, Flatten, Dropout, Convolution2D, Activation, MaxPooling2D, GlobalAveragePooling2D, concatenate
from keras import losses
from tensorflow import keras

In [4]:
""" GLOBAL VARIABLES """
NUM_CLASSES = 4
EPOCHS = 10
INSTACK = 16
IMG_SIZE = (224, 224)
LEN_TRAIN = 85

In [ ]:
""" Data loading """

train_gen = utils.joinedGen(Train=True)
val_gen = utils.joinedGen(Val=True)

In [4]:
TVHI_trained = "/content/drive/MyDrive/TFLModel_1"
OF_trained = "/content/drive/MyDrive/OF_model"

In [21]:
resnet_base = keras.models.load_model(TVHI_trained)
BaseResNet = Model(resnet_base.inputs, resnet_base.layers[-4].output, name='resnet_base')
BaseResNet.trainable = False
BaseResNet.summary()

Model: "resnet_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_input (InputLaye  [(None, 224, 224, 3)]    0         
 r)                                                              
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 res_net18 (ResNet18)        (None, 40)                11211432  
                                                                 
Total params: 11,211,432
Trainable params: 0
Non-trainable params: 11,211,432
_________________________________________________________________


In [6]:
of_base = keras.models.load_model(OF_trained)
OF_Base = Model(of_base.inputs, of_base.layers[-9].output, name='of_base')
OF_Base.trainable = False
OF_Base.summary()

Model: "of_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_input (InputLayer)   [(None, 16, 224, 224, 3)  0         
                             ]                                   
                                                                 
 conv3d (Conv3D)             (None, 16, 224, 224, 32)  2624      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 8, 112, 112, 32)  0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 8, 112, 112, 32)  128       
 ormalization)                                                   
                                                                 
 conv3d_1 (Conv3D)           (None, 8, 112, 112, 32)   27680     
                                                           

In [19]:
""" Building Model """

#Baseline model to keep expanding based on example from: https://keras.io/guides/training_with_built_in_methods/#passing-data-to-multiinput-multioutput-models
def build_TS_model():

  #Set up basemodels, load weights, remove certain layers, check output sizes. If not the same, use 1D convolutions, fuse together dense layers using the concatenate. 
  #Freeze baseline models. Set new layers to trainable. Train. Save weights and model. 
  resnet_base = keras.models.load_model(TVHI_trained)
  BaseResNet = Model(resnet_base.inputs, resnet_base.layers[-4].output, name='resnet_base')
  BaseResNet.trainable = False
  BaseResNet.summary()

  of_base = keras.models.load_model(OF_trained)
  OF_Base = Model(of_base.inputs, of_base.layers[-9].output, name='of_base')
  OF_Base.trainable = False
  OF_Base.summary()

  x = concatenate([BaseResNet.output, OF_Base.output])

  x = Conv2D(filters=64, kernel_size=(3,3), padding='same', activation = 'relu')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D(pool_size=(2,2), strides=2)(x)
  
  x = Flatten()(x)
  x = Dense(256, activation = 'relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)

  x = Dense(64, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.2)(x)
  
  x = Dense(NUM_CLASSES, activation='softmax')(x)

  model = keras.Model(
      inputs=[OF_Base.input, BaseResNet.input], outputs=x)
  model.compile(loss='categorical crossentropy', optimizer='Adam', metrics=['accuracy'])
  model.summary()

  return model 


In [20]:
ts_model = build_TS_model()

Model: "resnet_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_input (InputLaye  [(None, 224, 224, 3)]    0         
 r)                                                              
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 res_net18 (ResNet18)        (None, 40)                11211432  
                                                                 
Total params: 11,211,432
Trainable params: 0
Non-trainable params: 11,211,432
_________________________________________________________________
Model: "of_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_input (InputLayer)   [(None, 16, 224, 224, 3)  0         
                          

ValueError: ignored

In [8]:
""" Plot the model """
keras.utils.plot_model(ts_model, "multi_input_and_output_model.png", show_shapes=True)

NameError: ignored

In [ ]:
""" Set up callback and perform training """
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

ts_history = ts_model.fit(train_gen,epochs= EPOCHS, steps_per_epoch = LEN_TRAIN, validation_data=val_gen, validation_steps=15, callbacks= callback, verbose=1)

In [ ]:
""" Plotting results """
utils.plotAccuracy("Accuracy Optical Flow model", ts_history.history['accuracy'], ts_history.history['val_accuracy'])
utils.plotLoss("Loss Optical Flow model", ts_history.history['loss'], ts_history.history['val_loss'])

In [ ]:
""" Save model and weights """
ts_model.save_weights('TS_weights.h5')
ts_model.save("TS_model", save_format = 'tf', include_optimizer=True)